In [ ]:
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy


In [ ]:
from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np

true_stand_data=np.load(r'.//data3//true_array.npy')
false_stand_data=np.load(r'.//data3//false_array.npy')

sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }


true_sample = []
false_sample = []
for sample_number in range(true_stand_data.shape[0]):
    true_sample.append(copy.deepcopy(np.concatenate((true_stand_data[sample_number,:,:,0], true_stand_data[sample_number,:,:,1],true_stand_data[sample_number,:,:,2]),axis=1)))
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample.append(copy.deepcopy(np.concatenate((false_stand_data[sample_number,:,:,0], false_stand_data[sample_number,:,:,1],false_stand_data[sample_number,:,:,2]),axis=1)))
    
combin_train_data = np.concatenate((true_sample,false_sample),axis=0)    
combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()


import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((combin_train_data, combin_target_data))


model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=7,return_sequences=True,input_shape=(110,21)),
    tf.keras.layers.GRU(units=21,return_sequences=True),
    tf.keras.layers.GRU(units=63,return_sequences=True),
    tf.keras.layers.GRU(units=189,return_sequences=True),
    tf.keras.layers.GRU(units=69),
    tr.keras.layers.Dropout(0.2)
    tf.keras.layers.Dense(12,input_shape=(69,),activation='relu'),
    tf.keras.layers.Dense(2,,activation='softmax'),
    ])

print(model.summary())

callbacks_list = [
    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=3 
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict.h5', 
    monitor='loss', 
    save_best_only=True) 
]


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#             loss='tf.keras.losses.BinaryCrossentropy(from_logits=True)',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.fit(train_dataset, epochs=15,class_weight=class_weight,callbacks=callbacks_list)
